In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular
import pickle

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, integrated_brier_score
from sksurv.preprocessing import OneHotEncoder

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class LiverCancerSurvivalPredictor:
    """간암 생존 예측 모델 클래스 (XAI 포함, CDSS 호환)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.scaler = None
        self.label_encoders = {}
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        self.holdout_patient = None  # CDSS 테스트용 환자
        
        print(f"🚀 간암 생존 예측 모델 초기화 (XAI + CDSS 호환)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 선택된 컬럼들
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            # 인구학적 변수
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            # 종양 병기
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            # 간암 특이적 지표
            'child_pugh_classification', 'ishak_fibrosis_score',
            # 종양 특성
            'tumor_grade', 'primary_diagnosis', 'tissue_or_organ_of_origin', 'site_of_resection_or_biopsy',
            # 질병 진행
            'residual_disease', 'classification_of_tumor',
            # 과거력
            'prior_malignancy', 'synchronous_malignancy', 'prior_treatment',
            # 치료 관련
            'treatments_pharmaceutical_treatment_type', 'treatments_pharmaceutical_treatment_or_therapy',
            'treatments_pharmaceutical_treatment_intent_type',
            'treatments_radiation_treatment_type', 'treatments_radiation_treatment_or_therapy',
            'treatments_radiation_treatment_intent_type',
            # 시간 관련
            'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산
        self.processed_df['duration'] = self.processed_df['days_to_death'].fillna(
            self.processed_df['days_to_last_follow_up']
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (self.processed_df['duration'].notna()) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (CDSS 호환)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # CDSS 테스트용 환자 1명 미리 분리
        print("🔄 CDSS 테스트용 환자 분리 중...")
        holdout_idx = self.processed_df.sample(n=1, random_state=42).index[0]
        self.holdout_patient = self.processed_df.loc[holdout_idx:holdout_idx].copy()
        remaining_df = self.processed_df.drop(holdout_idx).copy()
        
        print(f"   - CDSS 테스트 환자: {holdout_idx}")
        print(f"   - 모델 훈련용 데이터: {len(remaining_df)}명")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in remaining_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration']]
        
        X = remaining_df[feature_cols].copy()
        y_duration = remaining_df['duration'].values
        y_event = remaining_df['event'].values.astype(bool)
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 수치형 변수: 중앙값으로 대체
        if numerical_cols:
            self.num_imputer = SimpleImputer(strategy='median')  # self.로 저장
            X[numerical_cols] = self.num_imputer.fit_transform(X[numerical_cols])
            print(f"✅ Imputer 훈련 완료: {len(numerical_cols)}개 수치형 변수")
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들
        meaningful_unknown_cols = [
            'child_pugh_classification', 'ishak_fibrosis_score', 'ajcc_pathologic_stage',
            'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'tumor_grade', 'residual_disease', 'prior_malignancy',
            'synchronous_malignancy', 'prior_treatment'
        ]
        
        self.label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            self.label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 다중공선성 해결
        print("\n🔍 다중공선성 검사 및 해결:")
        
        # 1. 분산이 0인 특성 제거
        zero_var_features = [col for col in X.columns if X[col].nunique() <= 1]
        if zero_var_features:
            print(f"🗑️  분산 0인 특성 제거: {zero_var_features}")
            X = X.drop(columns=zero_var_features)
        
        # 2. 높은 상관관계 특성 제거
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        high_corr_features = [column for column in upper.columns if any(upper[column] > 0.9)]
        
        if high_corr_features:
            print(f"🗑️  높은 상관관계 특성 제거 (>0.9): {high_corr_features}")
            X = X.drop(columns=high_corr_features)
        
        # 3. VIF 검사
        try:
            from statsmodels.stats.outliers_influence import variance_inflation_factor
            vif_data = pd.DataFrame()
            vif_data["특성"] = X.columns
            vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
            
            high_vif_features = vif_data[vif_data["VIF"] > 10]["특성"].tolist()
            if high_vif_features:
                print(f"🗑️  높은 VIF 특성 제거 (>10): {high_vif_features}")
                X = X.drop(columns=high_vif_features)
                
            print(f"✅ VIF 검사 완료")
        except Exception as e:
            print(f"⚠️  VIF 검사 건너뜀: {e}")
        
        print(f"📊 최종 특성 개수: {len(X.columns)}")
        
        # 특성 스케일링
        self.scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료 (CDSS 호환)")
        
        return X_scaled, y_structured, y_duration, y_event
    
    def split_data(self, X, y_structured, y_duration, y_event):
        """데이터 분할"""
        print("\n✂️  4. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event = \
            train_test_split(X, y_structured, y_duration, y_event, 
                           test_size=0.2, random_state=42, stratify=y_event)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event,
                           test_size=0.25, random_state=42, stratify=y_temp_event)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명)")
        print(f"📊 CDSS 테스트: 1명 (별도 보관)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event):
        """모델 훈련 (CDSS 호환)"""
        print("\n🤖 5. 모델 훈련")
        
        # 1. Cox 비례위험 모델 (scikit-survival)
        print("🔄 Cox 비례위험 모델 훈련 중...")
        try:
            cox_model = CoxPHSurvivalAnalysis(alpha=0.5)
            cox_model.fit(X_train, y_train_struct)
            self.models['Cox'] = cox_model
            print("✅ Cox 모델 훈련 완료")
        except Exception as e:
            print(f"❌ Cox 모델 훈련 실패: {e}")
        
        # 2. Random Survival Forest (CDSS 호환 버전)
        print("🔄 Random Survival Forest 훈련 중...")
        try:
            rsf_model = RandomSurvivalForest(
                n_estimators=100,
                max_depth=10,
                min_samples_split=10,
                min_samples_leaf=5,
                random_state=42,
                n_jobs=-1
            )
            rsf_model.fit(X_train, y_train_struct)
            
            # RSF 모델을 CDSS 호환 형태로 래핑
            rsf_wrapper = {
                'model': rsf_model,
                'model_type': 'RandomSurvivalForest',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['RSF'] = rsf_wrapper
            print("✅ Random Survival Forest 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ RSF 모델 훈련 실패: {e}")
        
        # 3. Gradient Boosting Survival Analysis
        print("🔄 Gradient Boosting Survival 훈련 중...")
        try:
            gbsa_model = GradientBoostingSurvivalAnalysis(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=3,
                random_state=42
            )
            gbsa_model.fit(X_train, y_train_struct)
            
            # GBSA 모델도 래핑
            gbsa_wrapper = {
                'model': gbsa_model,
                'model_type': 'GradientBoostingSurvivalAnalysis',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['GBSA'] = gbsa_wrapper
            print("✅ Gradient Boosting Survival 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ GBSA 모델 훈련 실패: {e}")
        
        # 4. Cox 모델 (lifelines)
        print("🔄 Lifelines Cox 모델 훈련 중...")
        try:
            train_data = X_train.copy()
            train_data['duration'] = y_train_dur
            train_data['event'] = y_train_event
            
            cph = CoxPHFitter(penalizer=0.5)
            cph.fit(train_data, duration_col='duration', event_col='event')
            
            # Lifelines Cox 모델도 래핑
            cox_lifelines_wrapper = {
                'model': cph,
                'model_type': 'CoxPHFitter',
                'feature_names': self.feature_names,
                'scaler': self.scaler,
                'label_encoders': self.label_encoders,
                'training_info': {
                    'n_samples': len(X_train),
                    'n_features': len(self.feature_names),
                    'event_rate': y_train_event.mean()
                }
            }
            
            self.models['Cox_lifelines'] = cox_lifelines_wrapper
            print("✅ Lifelines Cox 모델 훈련 완료 (CDSS 호환)")
        except Exception as e:
            print(f"❌ Lifelines Cox 모델 훈련 실패: {e}")
        
        print(f"\n🎯 총 {len(self.models)}개 모델 훈련 완료")
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성 (수정된 버전)"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 - Permutation Explainer 사용
        print("🔄 SHAP 설명 생성 중...")
        if 'RSF' in self.models:
            try:
                rsf_model = self.models['RSF']['model']
                X_test_sample = X_test.iloc[:50]
                
                # TreeExplainer 대신 Permutation Explainer 사용
                explainer = shap.PermutationExplainer(
                    rsf_model.predict, 
                    X_train.iloc[:100]  # 배경 데이터 샘플링
                )
                shap_values = explainer(X_test_sample)
                
                self.shap_explainers['RSF'] = explainer
                self.shap_values['RSF'] = shap_values.values
                print("✅ RSF SHAP 설명 생성 완료 (PermutationExplainer)")
            except Exception as e:
                print(f"❌ RSF SHAP 실패: {e}")
                # SHAP 실패 시 특성 중요도 대안 사용
                try:
                    # Permutation Importance 계산
                    from sklearn.inspection import permutation_importance
                    perm_importance = permutation_importance(
                        rsf_model, X_test_sample, 
                        [y_test_struct[i] for i in X_test_sample.index],
                        n_repeats=10, random_state=42
                    )
                    
                    self.permutation_importance = {
                        'importances': perm_importance.importances_mean,
                        'feature_names': self.feature_names
                    }
                    print("✅ Permutation Importance 계산 완료 (SHAP 대안)")
                except Exception as e2:
                    print(f"❌ Permutation Importance 실패: {e2}")
        
        # LIME 설명기 초기화 (이진 분류로 변환)
        print("\n🔄 LIME 설명 생성 중...")
        if 'RSF' in self.models:
            try:
                # 생존 예측을 이진 분류로 변환하는 함수
                def survival_to_binary_prob(X):
                    rsf_model = self.models['RSF']['model']
                    # 생존 함수 예측
                    surv_funcs = rsf_model.predict_survival_function(X)
                    # 5년(1825일) 생존 확률 계산
                    probs_5year = []
                    for surv_func in surv_funcs:
                        try:
                            prob_5year = surv_func(1825)  # 5년 생존 확률
                        except:
                            prob_5year = 0.5  # 기본값
                        probs_5year.append(prob_5year)
                    
                    # 이진 분류 확률로 변환 (생존/사망)
                    probs_5year = np.array(probs_5year)
                    return np.column_stack([1 - probs_5year, probs_5year])
                
                explainer = lime_tabular.LimeTabularExplainer(
                    training_data=X_train.values,
                    feature_names=self.feature_names,
                    class_names=['5년내 사망', '5년 생존'],
                    mode='classification',
                    discretize_continuous=True
                )
                
                self.lime_explainers['RSF'] = {
                    'explainer': explainer,
                    'predict_fn': survival_to_binary_prob
                }
                print("✅ RSF LIME 설명기 생성 완료")
            except Exception as e:
                print(f"❌ RSF LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 (SHAP 대안 포함)"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화 (가능한 경우)
        shap_figures = []
        if 'RSF' in self.shap_explainers:
            try:
                shap_vals = self.shap_values['RSF']
                
                # Summary plot
                plt.figure(figsize=(10,6))
                shap.summary_plot(shap_vals, X_test.iloc[:50], 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title("RSF 모델 특성 중요도 (SHAP)")
                shap_summary_path = "shap_summary_RSF.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                shap_figures.append(shap_summary_path)
                print("✅ SHAP 시각화 완료")
            except Exception as e:
                print(f"❌ SHAP 시각화 실패: {e}")
        
        # SHAP 실패 시 Permutation Importance 사용
        elif hasattr(self, 'permutation_importance'):
            try:
                plt.figure(figsize=(10,6))
                importance_df = pd.DataFrame({
                    'feature': self.permutation_importance['feature_names'],
                    'importance': self.permutation_importance['importances']
                }).sort_values('importance', ascending=True).tail(10)
                
                plt.barh(range(len(importance_df)), importance_df['importance'])
                plt.yticks(range(len(importance_df)), importance_df['feature'])
                plt.title("RSF 모델 특성 중요도 (Permutation Importance)")
                plt.xlabel("중요도")
                
                perm_importance_path = "permutation_importance_RSF.png"
                plt.savefig(perm_importance_path, bbox_inches='tight')
                plt.close()
                
                shap_figures.append(perm_importance_path)
                print("✅ Permutation Importance 시각화 완료")
            except Exception as e:
                print(f"❌ Permutation Importance 시각화 실패: {e}")
        
        # LIME 시각화
        lime_figures = []
        if 'RSF' in self.lime_explainers:
            try:
                lime_data = self.lime_explainers['RSF']
                exp = lime_data['explainer'].explain_instance(
                    X_test.iloc[sample_index].values,
                    lime_data['predict_fn'],
                    num_features=5
                )
                
                lime_path = f"lime_explanation_RSF_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"RSF 모델 LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                
                lime_figures.append(lime_path)
                print("✅ LIME 시각화 완료")
            except Exception as e:
                print(f"❌ LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event):
        """모델 평가"""
        print("\n📈 6. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event)
        }
        
        for model_name, model_wrapper in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event) in datasets.items():
                try:
                    if model_name == 'Cox':
                        # scikit-survival Cox 모델
                        risk_scores = model_wrapper.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                    
                    elif isinstance(model_wrapper, dict):
                        # 래핑된 모델들
                        actual_model = model_wrapper['model']
                        
                        if model_wrapper['model_type'] == 'CoxPHFitter':
                            # Lifelines Cox 모델
                            c_index = actual_model.concordance_index_
                        else:
                            # RSF, GBSA 모델
                            risk_scores = actual_model.predict(X)
                            c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                    
                    else:
                        # 기타 모델
                        risk_scores = model_wrapper.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                    
                    self.results[model_name][dataset_name] = {'c_index': c_index}
                    print(f"   {dataset_name}: C-index = {c_index:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'c_index': np.nan}
        
        return True
    
    def plot_survival_curves(self, X_test, y_test_dur, y_test_event):
        """생존 곡선 시각화 (XAI 포함)"""
        print("\n📊 7. 생존 곡선 시각화")
        
        fig = plt.figure(figsize=(25, 20))
        gs = fig.add_gridspec(3, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 전체 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 성별 생존 곡선
            fig.add_subplot(gs[0, 2]),  # 모델 성능 비교
            fig.add_subplot(gs[1, 0]),  # 특성 중요도 (Cox)
            fig.add_subplot(gs[1, 1]),  # 특성 중요도 (GBSA)
            fig.add_subplot(gs[1, 2]),  # CDSS 테스트 결과
            fig.add_subplot(gs[2, :])   # XAI 시각화
        ]
        
        fig.suptitle('간암 환자 생존 분석 결과 (XAI + CDSS 호환)', fontsize=16, fontweight='bold')
        
        # 1. 전체 Kaplan-Meier 생존 곡선
        kmf = KaplanMeierFitter()
        kmf.fit(y_test_dur, y_test_event, label='전체 환자')
        kmf.plot_survival_function(ax=axes[0])
        axes[0].set_title('전체 환자 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].grid(True, alpha=0.3)
        
        # 2. 성별에 따른 생존 곡선
        if 'gender' in self.processed_df.columns:
            test_indices = X_test.index
            gender_data = self.processed_df.loc[test_indices, 'gender'] if 'gender' in self.processed_df.columns else None
            
            if gender_data is not None:
                for gender in gender_data.unique():
                    mask = (gender_data == gender)
                    if mask.sum() > 5:
                        kmf_gender = KaplanMeierFitter()
                        kmf_gender.fit(y_test_dur[mask], y_test_event[mask], label=f'Gender: {gender}')
                        kmf_gender.plot_survival_function(ax=axes[1])
                
                axes[1].set_title('성별에 따른 생존 곡선')
                axes[1].set_ylabel('생존 확률')
                axes[1].set_xlabel('시간 (일)')
                axes[1].grid(True, alpha=0.3)
                axes[1].legend()
        
        # 3. 모델 성능 비교
        model_names = list(self.results.keys())
        test_c_indices = []
        for name in model_names:
            c_index = self.results[name]['Test']['c_index']
            if not np.isnan(c_index):
                test_c_indices.append(c_index)
            else:
                test_c_indices.append(0)
        
        bars = axes[2].bar(model_names, test_c_indices, 
                          color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
        axes[2].set_title('모델별 C-index 성능 비교 (테스트 세트)')
        axes[2].set_ylabel('C-index')
        axes[2].set_ylim(0.5, 1.0)
        axes[2].grid(True, alpha=0.3, axis='y')
        
        for bar, value in zip(bars, test_c_indices):
            if value > 0:
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4-5. 특성 중요도
        for idx, model_name in enumerate(['Cox_lifelines', 'GBSA']):
            ax_idx = 3 + idx
            if model_name in self.models:
                try:
                    if model_name == 'Cox_lifelines':
                        # Cox 계수 기반 중요도
                        cox_model = self.models[model_name]['model']
                        coefficients = cox_model.params_
                        importance_values = np.abs(coefficients.values)
                        feature_names_cox = coefficients.index.tolist()
                        
                        feature_importance_df = pd.DataFrame({
                            'feature': feature_names_cox,
                            'importance': importance_values
                        }).sort_values('importance', ascending=True).tail(10)
                        
                        color = 'lightcoral'
                    else:
                        # GBSA 특성 중요도
                        gbsa_model = self.models[model_name]['model']
                        if hasattr(gbsa_model, 'feature_importances_'):
                            importance = gbsa_model.feature_importances_
                            feature_importance_df = pd.DataFrame({
                                'feature': self.feature_names,
                                'importance': importance
                            }).sort_values('importance', ascending=True).tail(10)
                            color = 'lightgreen'
                        else:
                            continue
                    
                    bars = axes[ax_idx].barh(range(len(feature_importance_df)), 
                                           feature_importance_df['importance'],
                                           color=color)
                    
                    axes[ax_idx].set_yticks(range(len(feature_importance_df)))
                    axes[ax_idx].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                    axes[ax_idx].set_title(f'특성 중요도 ({model_name})', fontsize=12, fontweight='bold')
                    axes[ax_idx].set_xlabel('중요도')
                    axes[ax_idx].grid(True, alpha=0.3, axis='x')
                    
                except Exception as e:
                    print(f"❌ {model_name} 특성 중요도 시각화 실패: {e}")
        
        # 6. CDSS 테스트 결과
        self.test_cdss_compatibility(axes[5])
        
        # 7. XAI 시각화 로드
        try:
            img = plt.imread("shap_summary_RSF.png")
            axes[6].imshow(img)
            axes[6].axis('off')
            axes[6].set_title('SHAP 전역 설명 (RSF 모델)', fontsize=12)
        except Exception as e:
            axes[6].text(0.5, 0.5, 'SHAP 시각화 불러오기 실패', 
                        ha='center', va='center', transform=axes[6].transAxes,
                        fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        save_path = "liver_cancer_survival_analysis_xai_cdss.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"📁 시각화 결과 저장: {save_path}")
        
        return True
    
    def test_cdss_compatibility(self, ax):
        """CDSS 호환성 테스트 (수정된 버전)"""
        print("\n🔬 CDSS 호환성 테스트")
        
        try:
            # holdout 환자 데이터 전처리
            holdout_features = self.preprocess_holdout_patient()
            
            print(f"🔍 전처리된 특성 형태: {holdout_features.shape}")
            print(f"🔍 전처리된 특성명: {list(holdout_features.columns)}")
            
            # 각 모델로 예측 수행
            predictions = {}
            for model_name, model_wrapper in self.models.items():
                try:
                    print(f"\n🔄 {model_name} 모델 예측 중...")
                    
                    if model_name == 'Cox':
                        # scikit-survival Cox 모델
                        pred = model_wrapper.predict(holdout_features)[0]
                        
                    elif isinstance(model_wrapper, dict):
                        # 래핑된 모델들
                        actual_model = model_wrapper['model']
                        model_type = model_wrapper['model_type']
                        
                        print(f"   - 모델 타입: {model_type}")
                        print(f"   - 입력 특성 수: {holdout_features.shape[1]}")
                        
                        if model_type == 'CoxPHFitter':
                            # Lifelines Cox 모델 - DataFrame 형태로 예측
                            holdout_df = holdout_features.copy()
                            pred = actual_model.predict_partial_hazard(holdout_df).values[0]
                        else:
                            # RSF, GBSA 모델
                            pred = actual_model.predict(holdout_features)[0]
                    else:
                        # 기타 모델
                        pred = model_wrapper.predict(holdout_features)[0]
                    
                    predictions[model_name] = pred
                    print(f"✅ {model_name}: 예측값 = {pred:.4f}")
                    
                except Exception as e:
                    print(f"❌ {model_name} 예측 실패: {e}")
                    predictions[model_name] = np.nan
            
            # 결과 시각화
            valid_predictions = {k: v for k, v in predictions.items() if not np.isnan(v)}
            
            if valid_predictions:
                model_names = list(valid_predictions.keys())
                pred_values = list(valid_predictions.values())
                
                bars = ax.bar(model_names, pred_values, 
                            color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
                ax.set_title('CDSS 호환성 테스트\n(Holdout 환자 예측)')
                ax.set_ylabel('위험도 점수')
                ax.grid(True, alpha=0.3, axis='y')
                
                for bar, value in zip(bars, pred_values):
                    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
                
                print("✅ CDSS 호환성 테스트 완료")
            else:
                ax.text(0.5, 0.5, 'CDSS 테스트 실패\n모든 모델 예측 오류', 
                    ha='center', va='center', transform=ax.transAxes,
                    fontsize=12, fontweight='bold')
                    
        except Exception as e:
            print(f"❌ CDSS 호환성 테스트 실패: {e}")
            import traceback
            traceback.print_exc()
            ax.text(0.5, 0.5, f'CDSS 테스트 오류\n{str(e)[:50]}...', 
                ha='center', va='center', transform=ax.transAxes,
                fontsize=10, fontweight='bold')
    
    def preprocess_holdout_patient(self):
        """Holdout 환자 데이터 전처리 (특성명 완전 정렬)"""
        print("\n🔧 Holdout 환자 전처리 시작")
        
        # 1. 원본 holdout 환자 데이터에서 모든 특성 추출
        all_feature_cols = [col for col in self.holdout_patient.columns 
                        if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                        'event', 'duration']]
        
        patient_raw = self.holdout_patient[all_feature_cols].copy()
        print(f"🔍 원본 환자 특성: {len(patient_raw.columns)}개")
        print(f"🔍 모델 훈련 특성: {len(self.feature_names)}개")
        
        # 2. 모델 훈련 시 사용한 특성명과 정확히 일치하는 DataFrame 생성
        patient_processed = pd.DataFrame(index=patient_raw.index)
        
        # 🔥 핵심: 훈련 시 사용한 특성 순서대로 정확히 생성
        for feature_name in self.feature_names:
            if feature_name in patient_raw.columns:
                # 특성이 존재하는 경우 복사
                patient_processed[feature_name] = patient_raw[feature_name].copy()
                print(f"✅ {feature_name}: 원본 데이터 사용")
            else:
                # 특성이 없는 경우 기본값 설정
                patient_processed[feature_name] = 0.0  # float 타입으로 설정
                print(f"⚠️ {feature_name}: 기본값(0.0) 설정")
        
        # 3. 범주형 변수 전처리 (훈련 시와 동일한 방식)
        print("\n🔄 범주형 변수 전처리:")
        for col, encoder in self.label_encoders.items():
            if col in patient_processed.columns:
                try:
                    original_value = patient_processed[col].iloc[0]  # 수정: .iloc[0] 추가
                    print(f"   - {col}: 원본값 = {original_value}")
                    
                    # 문자열 처리
                    if pd.isna(original_value) or original_value == 'NA':
                        # 결측치는 첫 번째 클래스로 대체
                        patient_processed[col] = encoder.classes_[0]  # 수정: [0] 추가
                        print(f"     → 결측치를 '{encoder.classes_[0]}'로 대체")
                    else:
                        # 문자열로 변환 후 인코딩
                        str_value = str(original_value)
                        if str_value in encoder.classes_:
                            patient_processed[col] = encoder.transform([str_value])[0]  # 수정: [0] 추가
                            print(f"     → 인코딩: '{str_value}' → {patient_processed[col].iloc[0]}")
                        else:
                            # 새로운 카테고리는 첫 번째 클래스로 대체
                            patient_processed[col] = encoder.transform([encoder.classes_[0]])[0]  # 수정
                            print(f"     → 새로운 값 '{str_value}'을 '{encoder.classes_[0]}'로 대체")
                            
                except Exception as e:
                    print(f"     ❌ {col} 인코딩 실패: {e}")
                    patient_processed[col] = 0.0
        
        # 4. 🔥 중요: 모든 컬럼을 수치형으로 변환
        print("\n🔢 데이터 타입 변환:")
        for col in patient_processed.columns:
            try:
                patient_processed[col] = pd.to_numeric(patient_processed[col], errors='coerce')
                if patient_processed[col].isnull().any():
                    patient_processed[col] = patient_processed[col].fillna(0.0)
            except Exception as e:
                print(f"⚠️ {col} 수치형 변환 실패: {e}")
                patient_processed[col] = 0.0
        
        print(f"✅ 모든 컬럼 수치형 변환 완료")
        
        # 5. 🔥 핵심: 특성 순서를 훈련 시와 정확히 일치시키기
        patient_processed = patient_processed[self.feature_names]
        print(f"✅ 특성 순서 정렬 완료: {patient_processed.shape}")
        
        # 6. 수치형 변수 전처리 (이제 모든 컬럼이 수치형)
        if hasattr(self, 'num_imputer'):
            print(f"\n📏 Imputer 적용:")
            print(f"   - 입력 형태: {patient_processed.shape}")
            print(f"   - 입력 특성명: {list(patient_processed.columns)}")
            
            try:
                # 🔥 핵심: 특성명 정보 제거하고 numpy 배열로 변환
                patient_values = patient_processed.values
                imputed_values = self.num_imputer.transform(patient_values)
                
                # DataFrame으로 다시 변환
                patient_processed = pd.DataFrame(
                    imputed_values,
                    columns=self.feature_names,
                    index=patient_processed.index
                )
                print(f"✅ Imputer 적용 완료")
            except Exception as e:
                print(f"⚠️ Imputer 적용 실패, 건너뜀: {e}")
        
        # 7. 스케일링 적용
        print("\n📏 스케일링 적용:")
        try:
            # 🔥 핵심: 특성명 정보 제거하고 numpy 배열로 변환
            patient_values = patient_processed.values
            scaled_values = self.scaler.transform(patient_values)
            
            # DataFrame으로 다시 변환
            patient_features_scaled = pd.DataFrame(
                scaled_values,
                columns=self.feature_names,  # 정확한 특성명 사용
                index=patient_processed.index
            )
            
            print(f"✅ 최종 특성 형태: {patient_features_scaled.shape}")
            print(f"✅ 최종 특성명 일치: {list(patient_features_scaled.columns) == self.feature_names}")
            
            return patient_features_scaled
            
        except Exception as e:
            print(f"❌ 스케일링 실패: {e}")
            # 스케일링 실패 시 원본 반환
            return patient_processed
    
    def save_models_for_cdss(self):
        """CDSS 호환 모델 저장"""
        print("\n💾 CDSS 호환 모델 저장")
        
        # 전체 파이프라인을 하나의 객체로 저장
        cdss_pipeline = {
            'models': self.models,
            'scaler': self.scaler,
            'label_encoders': self.label_encoders,
            'feature_names': self.feature_names,
            'holdout_patient': self.holdout_patient,
            'preprocessing_info': {
                'meaningful_unknown_cols': [
                    'child_pugh_classification', 'ishak_fibrosis_score', 'ajcc_pathologic_stage',
                    'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
                    'tumor_grade', 'residual_disease', 'prior_malignancy',
                    'synchronous_malignancy', 'prior_treatment'
                ]
            },
            'metadata': {
                'created_date': datetime.now().isoformat(),
                'model_version': '1.0',
                'description': 'TCGA-LIHC 간암 생존 예측 모델 (CDSS 호환)'
            }
        }
        
        # 개별 모델도 저장
        for model_name, model_wrapper in self.models.items():
            try:
                filename = f"cdss_liver_cancer_{model_name.lower()}_model.pkl"
                with open(filename, 'wb') as f:
                    pickle.dump(model_wrapper, f)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        # 전체 파이프라인 저장
        try:
            pipeline_filename = "cdss_liver_cancer_complete_pipeline.pkl"
            with open(pipeline_filename, 'wb') as f:
                pickle.dump(cdss_pipeline, f)
            print(f"✅ 전체 파이프라인 저장: {pipeline_filename}")
        except Exception as e:
            print(f"❌ 파이프라인 저장 실패: {e}")
        
        return True
    
    def generate_report(self):
        """결과 보고서 생성"""
        print("\n📋 8. 결과 보고서 생성")
        
        report = []
        report.append("="*60)
        report.append("간암 환자 생존 예측 모델 분석 결과 (XAI + CDSS 호환)")
        report.append("="*60)
        report.append(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"데이터 경로: {self.data_path}")
        report.append("")
        
        # 데이터 요약
        report.append("📊 데이터 요약")
        report.append("-" * 30)
        report.append(f"총 환자 수: {len(self.processed_df) + 1}")  # holdout 포함
        report.append(f"모델 훈련용: {len(self.processed_df)}명")
        report.append(f"CDSS 테스트용: 1명")
        report.append(f"사망률: {self.processed_df['event'].mean()*100:.1f}%")
        report.append(f"중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        report.append(f"사용된 특성 수: {len(self.feature_names)}")
        report.append("")
        
        # 모델 성능
        report.append("🤖 모델 성능 (C-index)")
        report.append("-" * 30)
        
        for model_name in self.results:
            report.append(f"\n{model_name}:")
            for dataset in ['Train', 'Validation', 'Test']:
                if dataset in self.results[model_name]:
                    c_index = self.results[model_name][dataset]['c_index']
                    if not np.isnan(c_index):
                        report.append(f"  {dataset}: {c_index:.3f}")
        
        # 최고 성능 모델
        test_performances = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test']['c_index']
                if not np.isnan(c_index):
                    test_performances[model_name] = c_index
        
        if test_performances:
            best_model = max(test_performances, key=test_performances.get)
            report.append(f"\n🏆 최고 성능 모델: {best_model} (C-index: {test_performances[best_model]:.3f})")
        
        report.append("")
        report.append("ℹ️  특징:")
        report.append("   - CDSS 호환 모델 저장")
        report.append("   - XAI 설명 가능성 포함")
        report.append("   - Holdout 환자로 실제 예측 테스트")
        report.append("   - 모든 전처리 파이프라인 저장")
        report.append("")
        report.append("="*60)
        
        # 보고서 저장
        report_text = "\n".join(report)
        with open("liver_cancer_survival_xai_cdss_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        
        print("📁 보고서 저장: liver_cancer_survival_xai_cdss_report.txt")
        print("\n" + report_text)
        
        return report_text
    
    def run_complete_analysis(self):
        """전체 분석 실행 (XAI + CDSS 호환)"""
        print("🎯 간암 생존 예측 모델 전체 분석 시작 (XAI + CDSS 호환)")
        
        try:
            # 1. 데이터 로드
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비 (CDSS 호환)
            X, y_structured, y_duration, y_event = self.prepare_features()
            
            # 4. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event) = self.split_data(X, y_structured, y_duration, y_event)
            
            # 5. 모델 훈련 (CDSS 호환)
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event):
                return False
            
            # 6. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event):
                return False
            
            # 7. XAI 설명 생성
            self.explain_models(X_train, X_test)
            
            # 8. XAI 시각화 생성
            self.generate_xai_visualizations(X_test)
            
            # 9. 시각화 (XAI + CDSS 포함)
            if not self.plot_survival_curves(X_test, y_test_dur, y_test_event):
                return False
            
            # 10. 보고서 생성
            self.generate_report()
            
            # 11. CDSS 호환 모델 저장
            self.save_models_for_cdss()
            
            print("\n🎉 전체 분석 완료!")
            print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 실행
if __name__ == "__main__":
    # 데이터 경로
    data_path = r"C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv"
    
    # 분석 실행
    predictor = LiverCancerSurvivalPredictor(data_path)
    success = predictor.run_complete_analysis()
    
    if success:
        print("\n✨ 모든 분석이 성공적으로 완료되었습니다!")
        print("📁 생성된 파일들:")
        print("   - liver_cancer_survival_analysis_xai_cdss.png (시각화 결과)")
        print("   - liver_cancer_survival_xai_cdss_report.txt (분석 보고서)")
        print("   - cdss_liver_cancer_*_model.pkl (CDSS 호환 개별 모델)")
        print("   - cdss_liver_cancer_complete_pipeline.pkl (전체 파이프라인)")
        print("   - shap_*.png (SHAP 설명)")
        print("   - lime_*.png (LIME 설명)")
        print("\n🔬 CDSS 호환성:")
        print("   - 모든 전처리 파이프라인 포함")
        print("   - Holdout 환자로 실제 예측 테스트 완료")
        print("   - 모델 래핑으로 numpy 배열 오류 해결")
    else:
        print("\n❌ 분석 중 문제가 발생했습니다. 로그를 확인해주세요.")


✅ 한글 폰트 설정 완료
🚀 간암 생존 예측 모델 초기화 (XAI + CDSS 호환)
📁 데이터 경로: C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv
⏰ 시작 시간: 2025-06-14 16:41:47
🎯 간암 생존 예측 모델 전체 분석 시작 (XAI + CDSS 호환)

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 377행 × 87열
📈 데이터 기본 정보:
   - 총 환자 수: 377
   - 총 컬럼 수: 87
   - 생존 환자: 245명
   - 사망 환자: 132명
   - 사망률: 35.0%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 29개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 372명
   - 사망 이벤트: 132건
   - 중간 생존 시간: 602일

📋 결측값 분석:
   - days_to_death: 240개 (64.5%)
   - age_at_diagnosis: 3개 (0.8%)
   - ajcc_pathologic_stage: 24개 (6.5%)
   - ajcc_pathologic_t: 2개 (0.5%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - child_pugh_classification: 47개 (12.6%)
   - ishak_fibrosis_score: 96개 (25.8%)
   - tumor_grade: 5개 (1.3%)
   - residual_disease: 7개 (1.9%)
   - treatments_pharmaceutical_treatment_type: 42개 (11.3%)
   - treatments_pharmaceutical_treatment_or_therapy: 42개 (11.3%)
   - treatments_pharmaceutical_treatment_intent_type: 44개 (11.8%)
   - treatments_radiation_treatment_type: 2개 (0.5%)
  

PermutationExplainer explainer: 51it [00:52,  1.29s/it]                        


✅ RSF SHAP 설명 생성 완료 (PermutationExplainer)

🔄 LIME 설명 생성 중...
✅ RSF LIME 설명기 생성 완료

📊 XAI 시각화 생성
✅ SHAP 시각화 완료
✅ LIME 시각화 완료

📊 7. 생존 곡선 시각화

🔬 CDSS 호환성 테스트

🔧 Holdout 환자 전처리 시작
🔍 원본 환자 특성: 26개
🔍 모델 훈련 특성: 16개
✅ gender: 원본 데이터 사용
✅ race: 원본 데이터 사용
✅ ajcc_pathologic_stage: 원본 데이터 사용
✅ ajcc_pathologic_t: 원본 데이터 사용
✅ ajcc_pathologic_n: 원본 데이터 사용
✅ ajcc_pathologic_m: 원본 데이터 사용
✅ child_pugh_classification: 원본 데이터 사용
✅ ishak_fibrosis_score: 원본 데이터 사용
✅ tumor_grade: 원본 데이터 사용
✅ residual_disease: 원본 데이터 사용
✅ prior_malignancy: 원본 데이터 사용
✅ synchronous_malignancy: 원본 데이터 사용
✅ prior_treatment: 원본 데이터 사용
✅ treatments_pharmaceutical_treatment_or_therapy: 원본 데이터 사용
✅ treatments_radiation_treatment_type: 원본 데이터 사용
✅ treatments_radiation_treatment_or_therapy: 원본 데이터 사용

🔄 범주형 변수 전처리:
   - gender: 원본값 = male
     → 인코딩: 'male' → 1
   - race: 원본값 = asian
     → 인코딩: 'asian' → 1
   - ajcc_pathologic_stage: 원본값 = Stage II
     → 인코딩: 'Stage II' → 1
   - ajcc_pathologic_t: 원본값 = T2
     → 인코딩: 'T2' → 1
   -